### Advent of code 2018

"We've detected some temporal anomalies," one of Santa's Elves at the Temporal Anomaly Research and Detection Instrument Station tells you. She sounded pretty worried when she called you down here. "At 500-year intervals into the past, someone has been changing Santa's history!"

"The good news is that the changes won't propagate to our time stream for another 25 days, and we have a device" - she attaches something to your wrist - "that will let you fix the changes with no such propagation delay. It's configured to send you 500 years further into the past every few days; that was the best we could do on such short notice."

"The bad news is that we are detecting roughly fifty anomalies throughout time; the device will indicate fixed anomalies with stars. The other bad news is that we only have one device and you're the best person for the job! Good lu--" She taps a button on the device and you suddenly feel like you're falling. To save Christmas, you need to get all fifty stars by December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day in the advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

In [3]:
# first task is to open and read the input file
function read_file(file_path,dtype=nothing)
    # open, read in and close the file
    file = open(file_path)
    input = readlines(file)
    close(file)
    # change the data type if specified
    if dtype != nothing
        input = [parse(dtype,s) for s in input]
    end
    return input
end

read_file (generic function with 2 methods)

### Day 1: Chronal Calibration 

After feeling like you've been falling for a few minutes, you look at the device's tiny screen. "Error: Device must be calibrated before first use. Frequency drift detected. Cannot maintain destination lock." Below the message, the device shows a sequence of changes in frequency (your puzzle input). A value like +6 means the current frequency increases by 6; a value like -3 means the current frequency decreases by 3.

For example, if the device displays frequency changes of `+1, -2, +3, +1`, then starting from a frequency of zero, the following changes would occur:

- Current frequency  `0`, change of `+1`; resulting frequency  `1`.
- Current frequency  `1`, change of `-2`; resulting frequency `-1`.
- Current frequency `-1`, change of `+3`; resulting frequency  `2`.
- Current frequency  `2`, change of `+1`; resulting frequency  `3`.

In this example, the resulting frequency is 3.

Here are other example situations:

- `+1, +1, +1` results in `3`
- `+1, +1, -2` results in  `0`
- `-1, -2, -3` results in `-6`

Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?


Your puzzle answer was `500`.

The first half of this puzzle is complete! It provides one gold star: *

### Part Two 

You notice that the device repeats the same frequency change list over and over. To calibrate the device, you need to find the first frequency it reaches twice.

For example, using the same list of changes above, the device would loop as follows:

- Current frequency  `0`, change of `+1`; resulting frequency  `1`.
- Current frequency  `1`, change of `-2`; resulting frequency `-1`.
- Current frequency `-1`, change of `+3`; resulting frequency  `2`.
- Current frequency  `2`, change of `+1`; resulting frequency  `3`.

(At this point, the device continues from the start of the list.)

- Current frequency  `3`, change of `+1`; resulting frequency  `4`.
- Current frequency  `4`, change of `-2`; resulting frequency  `2`, which has already been seen.

In this example, the first frequency reached twice is 2. Note that your device might need to repeat its list of frequency changes many times before a duplicate frequency is found, and that duplicates might be found while in the middle of processing the list.

Here are other examples:

- `+1, -1` first reaches `0` twice.
- `+3, +3, +4, -2, -4` first reaches `10` twice.
- `-6, +3, +8, +5, -6` first reaches `5` twice.
- `+7, +7, -2, -7, -4` first reaches `14` twice.

What is the first frequency your device reaches twice?

Your puzzle answer was `709`.

Both parts of this puzzle are complete! They provide two gold stars: **

In [208]:
# first part: total calibration
function get_total_calibration(input,print=true)
    total_calibration = sum(input)
    if print == true
        println("The total calibration is: ", total_calibration)
    end
    return total_calibration
end

function find_repeat(input,print=true,verbose=true)
    # measure length of array
    array_length = size(input)[1]
    # set up starting values
    current_frequency = 0
    previous_frequencies = []
    # start at 1 (to get first value)
    counter = 0
    # loop through until we find a repeat value
    while in(current_frequency,previous_frequencies[1:1:end-1]) == false
        # add input value to frequency
        current_frequency += input[(counter % array_length) + 1]
        append!(previous_frequencies,current_frequency)
        # print to screen every N iterations
        if verbose == true
            if counter % 10000 == 0
                println("Done ", counter, " iterations")
            end
        end
        # add one to the counter
        counter += 1
    end
    # print out the value
    if print == true
        println("First repeat is ", current_frequency, 
                "on iteration ",counter-1)
    end
    return current_frequency, counter
end

find_repeat2 (generic function with 3 methods)

In [210]:
#input = read_file("Data/input1.txt",Int)
#total_calibration = get_total_calibration(input)
#freq, iteration = find_repeat(input,verbose=false)

### What have I learned?

Here, I can read in a file and convert it to an array in Julia. I can sum the array, and look for any repeat values.

For the second part, I used a simple `while` loop implementation to look for any repeat values. It may be worthwhile looking into vectorising any processes like this in the future.

### Day 2: Inventory Management System

You stop falling through time, catch your breath, and check the screen on the device. "Destination reached. Current Year: 1518. Current Location: North Pole Utility Closet 83N10." You made it! Now, to find those anomalies.

Outside the utility closet, you hear footsteps and a voice. "...I'm not sure either. But now that so many people have chimneys, maybe he could sneak in that way?" Another voice responds, "Actually, we've been working on a new kind of suit that would let him fit through tight spaces like that. But, I heard that a few days ago, they lost the prototype fabric, the design plans, everything! Nobody on the team can even seem to remember important details of the project!"

"Wouldn't they have had enough fabric to fill several boxes in the warehouse? They'd be stored together, so the box IDs should be similar. Too bad it would take forever to search the warehouse for two similar box IDs..." They walk too far away to hear any more.

Late at night, you sneak to the warehouse - who knows what kinds of paradoxes you could cause if you were discovered - and use your fancy wrist device to quickly scan every box and produce a list of the likely candidates (your puzzle input).

To make sure you didn't miss any, you scan the likely candidate boxes again, counting the number that have an ID containing exactly two of any letter and then separately counting those with exactly three of any letter. You can multiply those two counts together to get a rudimentary checksum and compare it to what your device predicts.

For example, if you see the following box IDs:

- `abcdef` contains no letters that appear exactly two or three times.
- `bababc` contains two a and three b, so it counts for both.
- `abbcde` contains two b, but no letter appears exactly three times.
- `abcccd` contains three c, but no letter appears exactly two times.
- `aabcdd` contains two a and two d, but it only counts once.
- `abcdee` contains two e.
- `ababab` contains three a and three b, but it only counts once.

Of these box IDs, four of them contain a letter which appears exactly twice, and three of them contain a letter which appears exactly three times. Multiplying these together produces a checksum of 4 * 3 = 12.

What is the checksum for your list of box IDs?

Your puzzle answer was `5658`.

The first half of this puzzle is complete! It provides one gold star: *

#### Part Two 

Confident that your list of box IDs is complete, you're ready to find the boxes full of prototype fabric.

The boxes will have IDs which differ by exactly one character at the same position in both strings. For example, given the following box IDs:

- `abcde`
- `fghij`
- `klmno`
- `pqrst`
- `fguij`
- `axcye`
- `wvxyz`

The IDs abcde and axcye are close, but they differ by two characters (the second and fourth). However, the IDs `fghij` and `fguij` differ by exactly one character, the third (h and u). Those must be the correct boxes.

What letters are common between the two correct box IDs? (In the example above, this is found by removing the differing character from either ID, producing `fgij`.)

Your puzzle answer was `nmgyjkpruszlbaqwficavxneo`.

Both parts of this puzzle are complete! They provide two gold stars: **

In [1]:
using StatsBase

function get_checksum(input,print=true)
    # initialise counters
    N_2_counts = 0
    N_3_counts = 0
    
    for item in input
        item_list = [char for char in item]
        # get counts for unique elements
        cm = countmap(item_list)
        N_2 = 0
        N_3 = 0
        # check if we have 2 or 3 repeat elements
        for key in keys(cm)
            if cm[key] == 2
                N_2 += 1
            elseif cm[key] == 3
                N_3 += 1
            end
        end
        # append counts to the dictionary
        N_2_counts += min(N_2,1)
        N_3_counts += min(N_3,1)
    end
    # print result to screen if specified
    if print == true
        println("  ",N_2_counts," for 2")
        println("+ ", N_3_counts," for 3")
        println("= ", N_3_counts * N_2_counts)
    end
    # return the count 
    return N_3_counts * N_2_counts
end

function find_matching_boxes(input,print=true)
    # initialise the parameters
    N_same_best = 0
    id0 = nothing
    id1 = nothing
    best_same = nothing
    id_list = nothing
    # loop through the input values
    for item0 in input
        for item1 in input
            item0_list = [char for char in item0]
            item1_list = [char for char in item1]
            if item0_list != item1_list
                same_values = [i == j for (i,j) in zip(item0_list,item1_list)]
                N_same = sum(same_values)
                # update values 
                if N_same > N_same_best
                    N_same_best = N_same
                    id0 = item0
                    id1 = item1
                    best_same = same_values
                    id_list = item0_list
                end
            end
        end
    end
    # print output to screen
    if print == true
        println("Matching boxes are ", id0, " and ", id1)
        println("Letters in common are ", join([i for i in id_list[best_same]]))
    end
    # return ids of the two matches
    return id0, id1
end  

find_matching_boxes (generic function with 2 methods)

In [4]:
#input = read_file("Data/input2.txt")
#checksum = get_checksum(input)
#id0, id1 = find_matching_boxes(input)

  246 for 2
+ 23 for 3
= 5658
Matching boxes are ndmgyjkpruszlbaqwficavxneo and nomgyjkpruszlbaqwficavxneo
Letters in common are nmgyjkpruszlbaqwficavxneo


("ndmgyjkpruszlbaqwficavxneo", "nomgyjkpruszlbaqwficavxneo")